In [83]:
import numpy as np
f1 = np.load("features/c001.npy")
f2 = np.load("features/c002.npy")


In [94]:
f2[1] @ f2[4]

0.9907575085947262

In [69]:
for key in data.keys():
    print(key, data[key].shape)

1 (869, 2560)
2 (3694, 2560)
3 (11588, 2560)
4 (13455, 2560)
5 (0,)
6 (164, 2560)
7 (14, 2560)
8 (1, 2560)
9 (0,)
10 (35, 2560)
11 (0,)
12 (10, 2560)
13 (7, 2560)
14 (19, 2560)
15 (11, 2560)
16 (0,)
17 (0,)
18 (0,)
19 (0,)
20 (2933, 2560)
21 (0,)
22 (0,)
23 (3618, 2560)
24 (107, 2560)
25 (1, 2560)
26 (0,)
27 (0,)
28 (933, 2560)
29 (28, 2560)
30 (3963, 2560)
31 (774, 2560)
32 (58, 2560)
33 (263, 2560)
34 (69, 2560)
35 (1044, 2560)
36 (0,)
37 (77, 2560)
38 (4037, 2560)
39 (212, 2560)
40 (187, 2560)
41 (2328, 2560)
42 (0,)
43 (2094, 2560)
44 (0,)
45 (0,)
46 (0,)
47 (3414, 2560)
48 (38, 2560)
49 (105, 2560)
50 (0,)
51 (0,)
52 (1, 2560)
53 (77, 2560)
54 (505, 2560)
55 (0,)
56 (1513, 2560)
57 (1262, 2560)
58 (0,)
59 (915, 2560)
60 (6744, 2560)
61 (0,)
62 (1618, 2560)
63 (4604, 2560)
64 (993, 2560)
65 (25, 2560)
66 (7, 2560)
67 (10, 2560)
68 (44, 2560)
69 (0,)
70 (7, 2560)
71 (17, 2560)
72 (0,)
73 (39, 2560)
74 (40, 2560)
75 (0,)
76 (500, 2560)
77 (0,)
78 (915, 2560)
79 (0,)
80 (1008, 2560)
8

In [18]:
import numpy as np
cos_sim = lambda x: np.dot(x[0], x[1]) / (np.linalg.norm(x[0]) * np.linalg.norm(x[1]))
cos_sim([data[1][0], data[1][5]])

0.9738754

# transform detects

In [38]:
import pandas as pd
detection_files = ["./detects/"+str(i)+".csv" for i in range(1, 8)]
for detection_file in detection_files:
    df = pd.read_csv(detection_file)
    df.rename(columns={"detect_id": "camera_track_id"}, inplace=True)
    df.rename(columns={"local_id": "detect_id"}, inplace=True)
    min_local_id = df["detect_id"].min()
    df["detect_id"] = df["detect_id"] - min_local_id
    df.set_index("detect_id", inplace=True)
    df.to_csv(detection_file)

# Transform groundtruth

In [39]:
import glob
for num_video in range(2, 7):
    gt_files = glob.glob("./groundtruth/"+str(num_video)+"/*.csv")
    for gt_file in gt_files:
        print(gt_file)
        vids = gt_file.split("groundtruth_")[-1].split(".csv")[0].split("_")
        gt_df = pd.read_csv(gt_file)
        gt_df.rename(columns={"global_object_id": "object_id"}, inplace=True)
        gt_df.rename(columns={"detect_id": "camera_track_id"}, inplace=True)
        all_video_gts = pd.DataFrame()
        for vid in vids:
            single_video_gt = gt_df[gt_df["camera_id"] == int(vid)]
            single_video_gt = single_video_gt[single_video_gt["object_id"] != -1]
            detect_df = pd.read_csv("./detects/"+str(vid)+".csv")
            single_video_gt = single_video_gt.merge(detect_df, on="camera_track_id")[["camera_id", "detect_id", "object_id"]]
            if vid == vids[0]:
                all_video_gts = single_video_gt
            else:
                all_video_gts = pd.concat([all_video_gts, single_video_gt])
        all_video_gts.to_csv(gt_file, index=False)

./groundtruth/2/groundtruth_4_5.csv
./groundtruth/2/groundtruth_2_3.csv
./groundtruth/2/groundtruth_1_2.csv
./groundtruth/2/groundtruth_5_6.csv
./groundtruth/2/groundtruth_3_4.csv
./groundtruth/2/groundtruth_6_7.csv
./groundtruth/3/groundtruth_3_4_5.csv
./groundtruth/3/groundtruth_4_5_6.csv
./groundtruth/3/groundtruth_5_6_7.csv
./groundtruth/3/groundtruth_2_3_4.csv
./groundtruth/3/groundtruth_1_2_3.csv
./groundtruth/4/groundtruth_1_2_3_4.csv
./groundtruth/4/groundtruth_2_3_4_5.csv
./groundtruth/4/groundtruth_3_4_5_6.csv
./groundtruth/4/groundtruth_4_5_6_7.csv
./groundtruth/5/groundtruth_1_2_3_4_5.csv
./groundtruth/5/groundtruth_3_4_5_6_7.csv
./groundtruth/5/groundtruth_2_3_4_5_6.csv
./groundtruth/6/groundtruth_2_3_4_5_6_7.csv
./groundtruth/6/groundtruth_1_2_3_4_5_6.csv


# transform features

In [81]:
from collections import defaultdict
from tqdm import tqdm
feature_files = [f"features/c00"+str(i)+".pkl" for i in range(1, 8)]
detection_files = ["./detects/"+str(i)+".csv" for i in range(1, 8)]
for feat_file, detect_file in zip(feature_files, detection_files):
    local_count = defaultdict(int)
    with open(feat_file, "rb") as f:
        features = pickle.load(f)
    new_features = []
    df = pd.read_csv(detect_file)

    # iterate over the detect data frame
    for idx, row in df.iterrows():
        local_id = row["camera_track_id"]
        feature_idx = local_count[local_id]
        if feature_idx >= len(features[local_id]):
            feature_idx = len(features[local_id]) - 1
        if feature_idx == -1:
            feat = np.zeros(2560)
        else:
            # normalize
            feat = features[local_id][feature_idx] / np.linalg.norm(features[local_id][feature_idx])
        detect_id = row["detect_id"]
        new_features.append([detect_id, feat])
        local_count[local_id] += 1
    new_features = sorted(new_features, key=lambda x: x[0])
    new_features = [x[1] for x in new_features]
    new_fueatures = np.array(new_features)
    np.save(feat_file.replace("pkl", "npy"), new_features)

In [78]:
x = sorted(new_features, key=lambda x: x[0])
x = [x[1] for x in x]
x = np.array(x)

In [80]:
x.shape

(244558, 2560)